In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [19]:
# scrape songs from artist page
# repeated usage (and usage of the other functions in the notebook on this Dataframe) will lead to azlyrics throwing up a captcha wall

response = requests.get("https://www.azlyrics.com/r/rem.html")
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")
song_lines = document.find_all("div", attrs={"class": "listalbum-item"})
artist = document.find("title").text.replace(" Lyrics", "")

songs_dictionary = {
    "artist": [],
    "title": [],
    "link": []
}
for line in song_lines:
    if line.find("a"):
        songs_dictionary["artist"].append(artist)
        songs_dictionary["title"].append(line.text)
        songs_dictionary["link"].append("https://www.azlyrics.com" + line.find("a").get("href"))
songs = pd.DataFrame(songs_dictionary)
songs

,artist,title,link
0,R.E.M.,Radio Free Europe,https://www.azlyrics.com/lyrics/rem/radiofreee...
1,R.E.M.,Pilgrimage,https://www.azlyrics.com/lyrics/rem/pilgrimage...
2,R.E.M.,Laughing,https://www.azlyrics.com/lyrics/rem/laughing.html
3,R.E.M.,Talk About The Passion,https://www.azlyrics.com/lyrics/rem/talkaboutt...
4,R.E.M.,Moral Kiosk,https://www.azlyrics.com/lyrics/rem/moralkiosk...
...,...,...,...
214,R.E.M.,Standing Still,https://www.azlyrics.com/lyrics/rem/standingst...
215,R.E.M.,The Great Beyond,https://www.azlyrics.com/lyrics/rem/thegreatbe...
216,R.E.M.,Wall Of Death,https://www.azlyrics.com/lyrics/rem/wallofdeat...
217,R.E.M.,"We All Go Back To Where We Belong(from ""Part L...",https://www.azlyrics.com/lyrics/rem/weallgobac...


In [33]:
# placeholder manually filled dataframe

data = {    
    "artist": ["R.E.M.", "R.E.M.", "R.E.M.", "R.E.M.", "R.E.M."],
    "title": ["Man On The Moon", "Nightswimming", "The Sidewinder Sleeps Tonite", "Everybody Hurts", "Monty Got A Raw Deal"],
    "link": ["https://www.azlyrics.com/lyrics/rem/manonthemoon.html", "https://www.azlyrics.com/lyrics/rem/nightswimming.html", "https://www.azlyrics.com/lyrics/rem/thesidewindersleepstonite.html", "https://www.azlyrics.com/lyrics/rem/everybodyhurts.html", "https://www.azlyrics.com/lyrics/rem/montygotarawdeal.html"]
}
songs = pd.DataFrame(data)
songs

,artist,title,link
0,R.E.M.,Man On The Moon,https://www.azlyrics.com/lyrics/rem/manonthemo...
1,R.E.M.,Nightswimming,https://www.azlyrics.com/lyrics/rem/nightswimm...
2,R.E.M.,The Sidewinder Sleeps Tonite,https://www.azlyrics.com/lyrics/rem/thesidewin...
3,R.E.M.,Everybody Hurts,https://www.azlyrics.com/lyrics/rem/everybodyh...
4,R.E.M.,Monty Got A Raw Deal,https://www.azlyrics.com/lyrics/rem/montygotar...


In [34]:
# scrape full html body

def html_scraper(link):
    response = requests.get(link)
    html_string = response.text
    return html_string
    
songs["html"] = songs["link"].apply(html_scraper)

songs

,artist,title,link,html
0,R.E.M.,Man On The Moon,https://www.azlyrics.com/lyrics/rem/manonthemo...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
1,R.E.M.,Nightswimming,https://www.azlyrics.com/lyrics/rem/nightswimm...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
2,R.E.M.,The Sidewinder Sleeps Tonite,https://www.azlyrics.com/lyrics/rem/thesidewin...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
3,R.E.M.,Everybody Hurts,https://www.azlyrics.com/lyrics/rem/everybodyh...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."
4,R.E.M.,Monty Got A Raw Deal,https://www.azlyrics.com/lyrics/rem/montygotar...,"<!DOCTYPE html>\r\n<html lang=""en"">\r\n<head>\..."


In [36]:
# get song text from full html body

def lyrics_scraper(html): 
    html_parsed = BeautifulSoup(html, "html.parser")
    main_div = html_parsed.find("div", attrs={"class": "col-xs-12 col-lg-8 text-center"})
    div_list = [div for div in main_div]
    lyrics_div = div_list[14]
    lyrics = lyrics_div.text
    return lyrics
    
songs["text"] = songs["html"].apply(lyrics_scraper)
songs.drop(columns = ["link", "html"], inplace = True)
songs

,artist,title,text
0,R.E.M.,Man On The Moon,\n\r\nMott the Hoople and the game of Life yea...
1,R.E.M.,Nightswimming,\n\r\nNightswimming deserves a quiet night\n\n...
2,R.E.M.,The Sidewinder Sleeps Tonite,\n\r\nThis here is the place where I will be s...
3,R.E.M.,Everybody Hurts,"\n\r\nWhen the day is long\nAnd the night, the..."
4,R.E.M.,Monty Got A Raw Deal,\n\r\nMonty this seems strange to me.\nThe mov...
